# Pandas Exercises
### By: Jingyu Li

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

## 3. Rewrite SQL in Pandas

### 3.1 Basics

In *salary* table, the salary of employees are recorded.
- Id: ID of the employee
- Name: Name of the employee
- Salary: Monthly salary of the employee
- DepartmentId: ID of the department that the employee belongs to

In *dp* table,
- Id: ID of the department
- Name: Name of the department

Rewrite the query to generate same table (DataFrame) by using Pandas

In [2]:
salary = pd.read_csv(r'data/salary.csv')
salary.head()

,Id,Name,Salary,DepartmentId
0,1,William,1600,1
1,2,James,1140,2
2,3,Harper,4788,2
3,4,Mason,6567,2
4,5,Evelyn,1112,1


In [3]:
dp = pd.read_csv(r'data/department.csv')
dp

,Id,Name
0,1,IT
1,2,DS
2,3,Sales
3,4,HR
4,5,Finance
5,6,Product


In [4]:
# Q48: get the second highest value of salary among all employees
'''
SELECT DISTINCT Salary AS secondhigh
FROM salary
ORDER BY Salary DESC
LIMIT 1 OFFSET 1
'''
res = salary['Salary'].unique()
res.sort()
res[-2]

9128

In [5]:
# Q49: query the Name and Salary of employees who satisfies the following conditions
# - salary is greater than 2000
# - in department 1 and 2
# order the results by Salary from low to high
'''
SELECT Name, Salary
FROM salary
WHERE Salary>2000 AND (DepartmentId=1 OR DepartmentId=2)
ORDER BY Salary
'''
res = salary.loc[(salary['Salary']>2000) & (salary['DepartmentId'].isin([1,2])), ['Name', 'Salary']]
res.sort_values(by=['Salary'])

,Name,Salary
2,Harper,4788
15,Lillian,6257
3,Mason,6567
11,Jack,7530
19,Alan,8747
5,Ella,9128
17,Aubrey,9817


In [6]:
# Q50: query the average salary of each department, the table contains two column:
# -name: name of the department
# -avg_salary: average salary of the department
# All departments in dp should be listed. If there aren't any employees belonging to a department in salary table,
# then avg_salary should be np.nan
# Order by average salary from high to low
'''
SELECT d.Name, sub.avg_salary
FROM dp d
LEFT JOIN (SELECT DepartmentId, AVG(Salary) AS avg_salary FROM salary GROUP BY DepartmentId) sub ON sub.DepartmentId=d.Id
ORDER BY sub.avg_salary DESC
'''
temp = salary.groupby('DepartmentId')['Salary'].mean()
res = dp.join(temp, on='Id', how='left')
res = res[['Name', 'Salary']]
res.columns = ['name', 'avg_salary']
res.sort_values(by=['avg_salary'], ascending=False)

,name,avg_salary
0,IT,4858.125000
1,DS,4081.800000
2,Sales,2269.250000
3,HR,1069.666667
4,Finance,NaN
5,Product,NaN


In [7]:
# Q51: return the employee's name whose id is in the following list: [1,3,7,9,14,15,19]
'''
SELECT Name
FROM salary
WHERE Id IN (1,3,7,9,14,15,19)
'''
salary.loc[salary['Id'].isin([1,3,7,9,14,15,19]), 'Name']

0      William
2       Harper
6        Avery
8     Scarlett
13     Eleanor
14     Grayson
18      Julian
Name: Name, dtype: object

In [8]:
# Q52: query the average salary of the department, in which the number of employees arenot less than 5
# the table contains three column:
# -Id: ID of the department
# -avg_salary: average salary of the department
# -employee_num: number of employees in that department
'''
SELECT DepartmentId AS Id, AVG(Salary) AS avg_salary, COUNT(Id) AS employee_num
FROM Salary
GROUP BY DepartmentId
HAVING COUNT(Id)>=5
'''
temp = salary.groupby('DepartmentId')['Salary'].agg(['mean','count']).reset_index()
temp.columns=['Id','avg_salary', 'employee_num']
temp = temp[temp['employee_num']>=5]
temp

,Id,avg_salary,employee_num
0,1,4858.125,8
1,2,4081.800,5


In [9]:
# Q53: query the top 3 Salaries of each department. Only the departments appear in salary table should be returned
# the table contains three column:
# -department: Name of the department
# -employee: Name of the employee
# -salary: salary of the employee
# Note: rank the salary using dense rank 
# e.g. a-1000, b-800, c-800, d-600, b and c both earn the second highest salary, d earn the third highest salary
# a, b, c and d should be all included
'''
WITH sub AS(
SELECT DepartmentId, Name AS employee, Salary AS salary,
       DENSE_RANK() OVER (PARTITION BY DepartmentId ORDER BY Salary DESC) AS ranks
FROM salary
)

SELECT d.Name AS department, sub.employee, sub.salary
FROM sub
LEFT JOIN dp d ON d.Id=sub.DepartmentId
WHERE sub.ranks <= 3
'''
def top_n(df, n=3, col='Salary'):
    ranks = df[col].rank(method='dense', ascending=False)
    return df[ranks<=3].sort_values(by=col, ascending=False)

temp = salary.groupby('DepartmentId',group_keys=False).apply(top_n)
temp = pd.merge(temp, dp, left_on='DepartmentId', right_on='Id', how='left', suffixes=('_em', '_dp'))
temp = temp[['Name_dp','Name_em','Salary']]
temp.columns = ['department', 'employee', 'salary']
temp

,department,employee,salary
0,IT,Aubrey,9817
1,IT,Ella,9128
2,IT,Alan,8747
3,DS,Mason,6567
4,DS,Lillian,6257
5,DS,Harper,4788
6,Sales,Scarlett,4536
7,Sales,Grayson,1772
8,Sales,Madison,1657
9,HR,Julian,1600


In *sales* table, the daily revenue from 2020-01-01 to 2020-02-29 of product 1, 2 and 3 are recorded. We can assume the dates in the table are continuous and there are no duplicates.
- Id: Product ID
- Date: Date
- Revenue: Daily revenue

Rewrite the query to generate same table (DataFrame) by using Pandas

In [10]:
sales = pd.read_csv(r'data/sales.csv')
sales.head()

,Id,Date,Revenue
0,3,1/19/2020,718
1,1,1/22/2020,690
2,2,1/22/2020,172
3,1,2/22/2020,139
4,2,2/22/2020,707


In [11]:
sales['Date'] = pd.to_datetime(sales['Date'])
df = sales.sort_values(by=['Id', 'Date'])
# sort on Id and Date, then we can work on df in the following problems

In [12]:
# Q54: query the cumulative revenue from day 1 for each product. Order the results by Id (1,2,3) and Date (low to high)
'''
SELECT Id, Date, SUM(Revenue) OVER (PARTITION BY Id ORDER BY Date) AS cum_revenue
FROM sales
ORDER BY Id, Date
'''
res = df.copy()
res['cum_revenue'] = res.groupby('Id')['Revenue'].cumsum()
res.drop('Revenue', axis=1, inplace=True)
res

,Id,Date,cum_revenue
31,1,2020-01-01,816
41,1,2020-01-02,1448
97,1,2020-01-03,2211
138,1,2020-01-04,3113
150,1,2020-01-05,3705
126,1,2020-01-06,4045
156,1,2020-01-07,4434
141,1,2020-01-08,5212
82,1,2020-01-09,5697
102,1,2020-01-10,6262


In [13]:
# Q55: return each product's total revenue in Jan and Feb of 2020 
'''
SELECT LEFT(Date, 7) AS month, Id, SUM(Revenue)
FROM sales
GROUP BY LEFT(Date, 7), Id
'''
res = df.copy()
res['Date'] = res['Date'].apply(lambda x: x.strftime('%Y-%m'))
res = res.groupby(['Date', 'Id'])['Revenue'].sum().reset_index()
res.columns = ['month', 'Id', 'revenue']
res

,month,Id,revenue
0,2020-01,1,18607
1,2020-01,2,16084
2,2020-01,3,16896
3,2020-02,1,14154
4,2020-02,2,13367
5,2020-02,3,13767


In [14]:
# Q56: return total daily revenue from 2020-01-15 to 2020-02-03 
'''
SELECT Date, SUM(Revenue)
FROM sales
WHERE Date BETWEEN '2020-01-15' AND '2020-02-03'
GROUP BY Date
'''
res = df.copy()
res = res.groupby('Date')['Revenue'].sum()
res = pd.DataFrame(res['2020-01-15':'2020-02-03'])
res

,Revenue
Date,
2020-01-15,1835
2020-01-16,1437
2020-01-17,1567
2020-01-18,1076
2020-01-19,1828
2020-01-20,1889
2020-01-21,1577
2020-01-22,1803
2020-01-23,1781


In [15]:
# Q57: return each product's total revenue on each day of the week. Columns of the table are:
# - id: product ID
# - monday: total revenue on Monday
# - tuesday: total revenue on Tuesday
# ......
# - sunday: total revenue on Sunday
'''
SELECT Id,
       SUM(CASE WHEN dow=2 THEN total ELSE 0 END) 'Monday',
       SUM(CASE WHEN dow=3 THEN total ELSE 0 END) 'Tuesday',
       SUM(CASE WHEN dow=4 THEN total ELSE 0 END) 'Wednesday',
       SUM(CASE WHEN dow=5 THEN total ELSE 0 END) 'Thursday',
       SUM(CASE WHEN dow=6 THEN total ELSE 0 END) 'Friday',
       SUM(CASE WHEN dow=7 THEN total ELSE 0 END) 'Saturday',
       SUM(CASE WHEN dow=1 THEN total ELSE 0 END) 'Sunday'
FROM
(
SELECT Id,
       DAYOFWEEK(Date) AS dow,
       SUM(Revenue) AS total
FROM sales
GROUP BY Id, dow) sub
GROUP BY Id
'''
res = df.copy()
dow = {'0':'Sundy','1':'Monday', '2':'Tuesday','3':'Wednesday','4':'Thursday','5':'Friday','6':'Saturday'}
res['dow'] = res['Date'].apply(lambda x: x.strftime("%w")).map(dow)
res = pd.DataFrame(res.groupby(['Id','dow'])['Revenue'].sum())
res.unstack()

Revenue                                                 
dow  Friday Monday Saturday Sundy Thursday Tuesday Wednesday
Id                                                          
1      4952   4647     4059  3747     4823    4299      6234
2      5292   4719     3503  2852     5188    4510      3387
3      4746   3602     3131  4443     6153    3079      5509

In [16]:
# Q58: return the rolling average of daily revenue in last 14 days
# - id: product ID
# - date: date
# - rolling_avg: average revenue in last 14 days
# Note: the first 13 days shouldn't be included in the final output
'''
SELECT Id, Date, AVG(Revenue) OVER (ORDER BY Date ROWS BETWEEN 13 PRECEDING AND 0 FOLLOWING) AS rolling_avg
FROM sales
WHERE Date >= (SELECT DISTINCT Date FROM sales ORDER BY Date LIMIT 1 OFFSET 13)
'''
def get_rolling(df):
    return df['Revenue'].rolling(14).mean()

res = df.copy()
res = res.set_index(['Id','Date'])
res = pd.DataFrame(res.groupby(level=0).apply(get_rolling))
res.index = res.index.droplevel(0)
res = res[res['Revenue'].notna()]
res.columns = ['rolling_avg']
res

rolling_avg
Id Date                   
1  2020-01-14   603.214286
   2020-01-15   608.285714
   2020-01-16   625.285714
   2020-01-17   612.214286
   2020-01-18   589.142857
   2020-01-19   575.928571
   2020-01-20   598.357143
   2020-01-21   607.785714
   2020-01-22   601.500000
   2020-01-23   589.928571
   2020-01-24   597.285714
   2020-01-25   633.857143
   2020-01-26   672.571429
   2020-01-27   607.857143
   2020-01-28   636.285714
   2020-01-29   597.071429
   2020-01-30   542.142857
   2020-01-31   558.928571
   2020-02-01   529.214286
   2020-02-02   506.857143
   2020-02-03   504.428571
   2020-02-04   505.000000
   2020-02-05   497.000000
   2020-02-06   522.714286
   2020-02-07   527.642857
   2020-02-08   482.785714
   2020-02-09   456.500000
   2020-02-10   495.571429
   2020-02-11   497.000000
   2020-02-12   513.571429
...                    ...
3  2020-01-31   596.357143
   2020-02-01   568.571429
   2020-02-02   583.500000
   2020-02-03   574.285714
   2020-02-04   595.214286
   2020-02-05   540.857143
   2020-02-06   517.714286
   2020-02-07   473.214286
   2020-02-08   473.785714
   2020-02-09   463.357143
   2020-02-10   454.500000
   2020-02-11   429.214286
   2020-02-12   448.000000
   2020-02-13   424.214286
   2020-02-14   425.214286
   2020-02-15   487.071429
   2020-02-16   428.214286
   2020-02-17   423.357143
   2020-02-18   399.500000
   2020-02-19   449.785714
   2020-02-20   493.571429
   2020-02-21   528.928571
   2020-02-22   491.571429
   2020-02-23   502.928571
   2020-02-24   504.642857
   2020-02-25   538.142857
   2020-02-26   506.428571
   2020-02-27   523.285714
   2020-02-28   538.357143
   2020-02-29   490.857143

[141 rows x 1 columns]

In [17]:
# Q59: calculate the daily changes in overall revenue of three products
'''
SELECT Date, daily_total/LAG(daily_total, 1) OVER (ORDER BY Date)-1 AS change_rate
FROM(
SELECT Date, SUM(Revenue) AS daily_total
FROM sales
GROUP BY Date
ORDER BY Date) sub
'''
res = df.copy()
res = pd.DataFrame(res.groupby('Date')['Revenue'].sum())
res['change_rate'] = res['Revenue']/res['Revenue'].shift(1)-1
res.drop('Revenue', axis=1, inplace=True)
res.head(10)

,change_rate
Date,
2020-01-01,NaN
2020-01-02,0.568966
2020-01-03,-0.218152
2020-01-04,-0.266528
2020-01-05,0.175302
2020-01-06,-0.203502
2020-01-07,0.219864
2020-01-08,0.461156
2020-01-09,-0.098256


### 3.2 Case: Game Play Analysis

This case contains a series of problems, which is modified from the SQL exercise in Leetcode.

In *activity* table, each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on some day using some device.
- player_id: ID of the player
- device_id: ID of the device
- event_date: Date of the record
- games_played: number of games played

Rewrite the query to generate same table (DataFrame) by using Pandas

In [18]:
activity = pd.read_csv(r'data/activity.csv')
activity.sample(5)

,player_id,device_id,event_date,games_played
498,21,185,2/16/2019,81
379,53,109,3/15/2019,89
777,54,91,3/25/2019,63
642,30,165,3/18/2019,14
393,70,25,3/8/2019,34


In [19]:
df = activity.copy()
df['event_date'] = pd.to_datetime(df['event_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
player_id       1000 non-null int64
device_id       1000 non-null int64
event_date      1000 non-null datetime64[ns]
games_played    1000 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 31.3 KB


In [20]:
# Q60: report the first login date for each player.
'''
SELECT player_id, MIN(event_date) AS first_login
FROM Activity
GROUP BY player_id
'''
res = df.copy()
res = pd.DataFrame(res.groupby('player_id')['event_date'].min())
res.columns = ['first_login']
res.head(10)

,first_login
player_id,
1,2019-01-01
2,2019-01-04
3,2019-01-13
4,2019-01-04
5,2019-01-12
6,2019-01-14
7,2019-01-04
8,2019-01-06
9,2019-01-04


In [21]:
# Q61: report the device that is first logged in for each player.
'''
SELECT player_id, device_id
FROM
(SELECT player_id, device_id, RANK() OVER (PARTITION BY player_id ORDER BY event_date) AS ranks FROM Activity) sub
WHERE ranks = 1
'''
res = df.copy()

def get_rank(df, n=1, col='event_date'):
    ranks = df[col].rank(method='dense')
    return df[ranks==1]

res = res.groupby('player_id')[['device_id','event_date']].apply(get_rank).reset_index()
res.drop(['level_1', 'event_date'], axis=1, inplace=True)
res.head(10)

,player_id,device_id
0,1,120
1,2,80
2,3,200
3,4,12
4,5,8
5,6,112
6,7,71
7,8,133
8,9,127
9,10,99


In [22]:
# Q62: report for each player and date, how many games played so far by the player. 
# That is, the total number of games played by the player until that date.
'''
SELECT a1.player_id, a1.event_date, SUM(a2.games_played) AS games_played_so_far
FROM Activity a1
LEFT JOIN Activity a2 ON a1.player_id=a2.player_id AND a1.event_date >= a2.event_date
GROUP BY a1.player_id, a1.event_date
ORDER BY a1.player_id, a1.event_date
'''
res = df.copy()
res = res.sort_values(by=['player_id', 'event_date'])

res = res.join(res.groupby('player_id')['games_played'].cumsum(), how='left', lsuffix='', rsuffix='_so_far')
res = res[['player_id', 'event_date', 'games_played_so_far']]
res.head(20)

,player_id,event_date,games_played_so_far
579,1,2019-01-01,40
94,1,2019-01-03,56
314,1,2019-01-06,139
688,1,2019-01-13,207
548,1,2019-01-27,213
438,1,2019-01-28,228
970,1,2019-02-15,246
610,1,2019-02-16,264
845,1,2019-02-21,272
75,1,2019-03-10,311


In [23]:
# Q63: report the fraction of players that logged in again on the day after the day they first logged in. 
# In other words, count the number of players that logged in for at least two consecutive days 
# starting from their first login date, then divide that number by the total number of players.
'''
WITH firstlog AS
(
SELECT player_id, event_date, RANK() OVER (PARTITION BY player_id ORDER BY event_date) AS ranks
FROM Activity
),

daytwo AS
(
SELECT a1.player_id
FROM firstlog a1, firstlog a2
WHERE a1.player_id=a2.player_id AND DATEDIFF(a2.event_date, a1.event_date) = 1 AND a1.ranks=1
)

SELECT ROUND(COUNT(DISTINCT daytwo.player_id)/COUNT(DISTINCT a.player_id),2) AS fraction
FROM daytwo, Activity a
'''
total = len(df['player_id'].unique())
res = df.copy()

login_2ndday = []
for player_id, data in res.groupby('player_id'):
    dates = data['event_date'].sort_values()
    if (dates.iloc[1]-dates.iloc[0]).days == 1:
        login_2ndday.append(player_id)

print(login_2ndday)
print(round(len(login_2ndday)/total,2))

[4, 9, 12, 19, 28, 39, 48, 59, 60, 67, 86, 98]
0.12


In [24]:
# Q64: define the install date of a player to be the first login day of that player.
# define day 1 retention of some date X to be the number of players whose install date is X and 
# they logged back in on the day right after X, divided by the number of players whose install date is X.
# report for each install date, the number of players that installed the game on that day and the day 1 retention.
# - install_dt: install date
# - installs: the number of players that installed the game on that day
# - d1_retention: day 1 retention rate of the install date
'''
SELECT a1.register_date AS install_dt,
       COUNT(a1.register_date) AS installs,
       ROUND(COUNT(a2.event_date) / COUNT(a1.register_date)*1.0,2) AS d1_retention
FROM 
    (SELECT player_id, MIN(event_date) AS register_date
     FROM Activity
     GROUP BY player_id) a1
LEFT JOIN Activity a2 ON a1.player_id = a2.player_id AND DATEDIFF(a2.event_date, a1.register_date) =1
GROUP BY a1.register_date
'''
res = df.copy()
res = pd.DataFrame(res.groupby('player_id')['event_date'].min())
res.columns = ['first_login']
res = res.reset_index()
install_count = res.groupby('first_login', as_index=False)['player_id'].count()
install_count.rename({'first_login':'install_dt', 'player_id':'installs'},axis=1,inplace=True)
retention = res[res['player_id'].isin(login_2ndday)]
retention_count = retention.groupby('first_login', as_index=False)['player_id'].count()
retention_count.rename({'first_login':'install_dt', 'player_id':'retention_count'},axis=1,inplace=True)

final = pd.merge(install_count, retention_count, how='left', on='install_dt')
final['d1_retention'] = final['retention_count'].div(final['installs'],fill_value=0)
final

,install_dt,installs,retention_count,d1_retention
0,2019-01-01,14,NaN,0.000000
1,2019-01-02,7,NaN,0.000000
2,2019-01-03,8,2.0,0.250000
3,2019-01-04,10,2.0,0.200000
4,2019-01-05,4,1.0,0.250000
5,2019-01-06,3,NaN,0.000000
6,2019-01-07,4,1.0,0.250000
7,2019-01-08,6,1.0,0.166667
8,2019-01-09,1,NaN,0.000000
9,2019-01-10,6,1.0,0.166667
